<img heigth="8" src="https://i.imgur.com/BhG5KQ3.png" alt="pbs-enae">

<h1 align="left">Visualización interactiva de datos con Python</h1>

<h2 align="left"><i>Creating Dashboards for Business Applications</i></h2>

<p align="left">
  <h3><a href="https://joefaver.dev">Joseph F. Vergel-Becerra</a> | Aplicaciones de Python - Tools and Skill Courses</h3>
  <br>
  <b>Last updated:</b> <i>28/04/2023</i>
  <br><br>
  <!-- <a href="#tabla-de-contenido">Tabla de contenido</a> • -->
  <a href="#referencias">Referencias</a> •
  <a href="#contribuir">Contribuir</a>
  <!-- <a href="#agradecimientos">Agradecimientos</a> -->
  <br><br>
</p>
<table align="left">
  <td>
      <a href="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000">
        <img src="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000" alt="Version" height="18">
      </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/joefavergel/pbs-enae-python-applications-course/blob/main/5-interactive-data-visualization-with-python/5-creating-dashboards-for-business-applications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
      </a>
  </td>
  <td>
    <a href="https://github.com/joefavergel/pbs-enae-python-applications-course" target="_parent"><img src="https://img.shields.io/github/forks/joefavergel/pbs-enae-ml-course?style=social" alt="Fork"/>
      </a>
  </td>
</table>
<br>
<br>

---

El ***Data Storytelling"*** es un enfoque de comunicación que combina técnicas narrativas con visualizaciones de datos para transmitir información de manera efectiva y atractiva. Su objetivo es facilitar la comprensión, retención e interpretación de información compleja, permitiendo a los receptores tomar decisiones informadas y basadas en datos. A través de las bibliotecas [plotly]() y [Dash]() es posible crear gráficos y aplicaciones web interactivas que te permitirán *contar historias sobre los datos*. Generar visualizaciones dinámicas de datos, personalizar estilos y diseño e integrar gráficos en aplicaciones web, le permite a las empresas tomar decisiones informadas basadas en el análisis de datos.

## 1. Preludio

In [1]:
!pip install --q dash==2.0.0 jupyter-dash==0.4.0 dash_bootstrap_components pandas;
!pip uninstall -y werkzeug
!pip install werkzeug==2.0.3

Found existing installation: Werkzeug 2.0.3
Uninstalling Werkzeug-2.0.3:
  Successfully uninstalled Werkzeug-2.0.3
  Using cached Werkzeug-2.0.3-py3-none-any.whl (289 kB)


## 2. Plotly

In [2]:
import plotly.io as pio


fig = dict({
    "data": [{"type": "bar",
              "x": [1, 2, 3],
              "y": [1, 3, 2]}],
    "layout": {"title": {"text": "A Figure Specified By Python Dictionary"}}
})

pio.show(fig)

In [3]:
import plotly.graph_objects as go

import pandas as pd

z_data = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv'
)

fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title='PBS Elevation', autosize=False,
                  width=800, height=800,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.show()

## 3. Dash y Jupyter-Dash

En clase el script de nuestro primer código en dash (`./my-apps/hello-dash.py`) se veía así:

![hello-dash](https://i.imgur.com/xqA2PYE.png)

En esta implementación de `jupyter-dash`, ahora es posible correr `dash`, ya no como un script, si no embebido en cuaderno de `jupyter-notebook`. En la siguiente celda se declaran los cambios y se mantienen comentadas las lineas de código que fueron sustituidas:

In [4]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash


# New external_stylesheets
external_stylesheets = [dbc.themes.BOOTSTRAP]

# app = dash.Dash()
app = JupyterDash(external_stylesheets=external_stylesheets)

app.layout = html.Div('Hello Dash!')

# New callback
@app.callback(Output('out', 'children'), Input('tbl', 'active_cell'))
def update_graphs(active_cell):
  return str(active_cell)


# if __name__ == '__main__':
#     app.run_server()
app.run_server(mode='inline')

/var/folders/yj/b8l2qgqd3_n2z00mzxsy1snc0000gn/T/ipykernel_76383/1169103883.py:3: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [5]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(external_stylesheets=external_stylesheets)


df = pd.read_csv(
    'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
)


@app.callback(
    Output('life-exp-vs-gdp', 'figure'),
    Input('year-slider', 'value')
)
def update_figure(selected_year):
    filterDf = df[df.year == selected_year]
    fig = px.scatter(
        filterDf, x="gdpPercap", y="lifeExp", size="pop", color="continent",
        hover_name="country", log_x=True, size_max=60
    )

    fig.update_layout(transition_duration=500)

    return fig


@app.callback(
    Output('year-pop', 'figure'),
    [dash.dependencies.Input('life-exp-vs-gdp', 'hoverData')]
)
def update_output_div(hoverData):
    if not hoverData:
        country = ''
    else:
        country = hoverData['points'][0]['hovertext']

    filterDf = df[df.country == country]
    fig = px.bar(filterDf, x='year', y='pop', title='Year Vs Population: {}'.format(country))
    # return 'Output: {}'.format(hoverData['points'][0]['hovertext'])
    return fig


app.layout = html.Div([
    dcc.Graph(id='life-exp-vs-gdp'),
    dcc.Slider(
        id='year-slider', min=df['year'].min(), value=df['year'].min(),
        max=df['year'].max(),
        marks={str(year): str(year) for year in df['year'].unique()},
        step=None
    ),
    dcc.Graph(id='year-pop'),
])

# New callback
@app.callback(Output('out', 'children'), Input('tbl', 'active_cell'))
def update_graphs(active_cell):
  return str(active_cell)


# if __name__ == '__main__':
#     app.run_server()
app.run_server(mode='inline')


/var/folders/yj/b8l2qgqd3_n2z00mzxsy1snc0000gn/T/ipykernel_76383/2562875877.py:3: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/Users/joefaver/anaconda3/envs/pbs/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



In [6]:
import dash
from dash import dcc, Input, Output
from jupyter_dash import JupyterDash
import dash_table as dt
import pandas as pd
import dash_bootstrap_components as dbc

df = pd.read_csv('https://git.io/Juf1t')

external_stylesheets = [dbc.themes.BOOTSTRAP]

app = JupyterDash(external_stylesheets=external_stylesheets)

app.layout = dbc.Container([
    dbc.Label('Click a cell in the table:'),
    dt.DataTable(
        id='tbl', data=df.to_dict('records'),
        columns=[{"name": i, "id": i} for i in df.columns],   
    ),
    dbc.Alert("Click the table", id='out'),
])

@app.callback(Output('out', 'children'), Input('tbl', 'active_cell'))
def update_graphs(active_cell):
  return str(active_cell)

app.run_server(mode='inline')

/var/folders/yj/b8l2qgqd3_n2z00mzxsy1snc0000gn/T/ipykernel_76383/3335509757.py:4: UserWarning:


The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`

/Users/joefaver/anaconda3/envs/pbs/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

